In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 7.6 MB/s 
     |████████████████████████████████| 358 kB 66.6 MB/s 
     |████████████████████████████████| 139 kB 55.4 MB/s 
     |████████████████████████████████| 55 kB 5.1 MB/s 
     |████████████████████████████████| 4.1 MB 49.1 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait,TimeoutException
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [5]:
url_target = 'https://www.idealista.com/venta-viviendas/madrid-madrid/'
driver.get(url_target)

In [ ]:
total_properties = int(driver.find_elements(By.CSS_SELECTOR,'span.breadcrumb-navigation-element-info')[-1])
total_properties

In [ ]:
total_pages = total_properties/30 # I know there are 30 properties per page max.
total_pages

In [ ]:
def get_properties_url_list(driver:webdriver,url:str) -> list:
  driver.get(url)
  
  url_list = []
  for page in range(total_pages):
    try:
      links = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.item-link')))
    except TimeoutException:
      print(f'failed to retrieve data from {url}')

    url_list.extend([link for link.href in links])
    siguiente_button = driver.find_element(by=By.LINK_TEXT,value='Siguiente')
    driver.get(siguiente_button.href)

  return url_list

In [ ]:
def get_property_data(driver:webdriver, url:str) -> pd.DataFrame:
  driver.get(url)
  utag_data = driver.execute_script('return window.utag_data')  # price, size, n_bedrooms, type(indirectly), floor, elevator, garden, terrace, energetic_class, garage, status

  data = {
      'price':int(utag_data.ad_price),
      'size':int(utag_data.ad_characteristics_constructedArea),
      'n_bedrooms':int(utag_data.ad_characteristics_bathNumber),
      'n_bathrooms':int(utag_data.ad_characteristics_roomNumber),
      'has_garage' : int(utag_data.ad_characteristics_hasParking),
      'good_condition': int(utag_data.ad_condition_isGoodCondition),
      'needs_renovating': int(utag_data.ad_condition_isNeedsRenovating),
      'new_development': int(utag_data.ad_condition_isNewDevelopment),
      'energy_cert':utag_data.get('ad_energyCertification_type','not_available'),
      'has_lift': int(utag_data.get('ad_characteristics_hasLift',0)),
      'has_garden': int(utag_data.get('ad_characteristics_hasGarden',0)),
      'has_terrace':int(utag_data.get('ad_characteristics_hasTerrace',0)),
      'has_swimming_pool': int(utag_data.get('ad_characteristics_hasSwimmingPool',0))  
  }

  if 'ad_characteristics_hasLift' in utag_data.leys():
    data['type'] = 'apartment'
  else:
    data['type'] = 'house'

  try:
    tags = driver.find_element(By.CSS_SELECTOR,'div.info-features-tags')
    data['luxury'] = ('lujo' in tags)
  except:
    data['luxury'] = False

    return pd.DataFrame(data).to_csv('idealista_real_estate_data.csv',mode='a')

In [ ]:
url_list = get_properties_url_list(driver,url_target)

for url in url_list:
  get_property_data(driver,url)